## Observaciones de codificación

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Revisión: ¿qué es una política?

- En RL intentamos aprender una política, ¿qué es esto exactamente?
- Una política asigna **observaciones** a **acciones**.
- En otras palabras, las observaciones son todo lo que la política "ve".

#### Políticas del lago aleatorio

- ¿Qué son las observaciones del lago aleatorio?
- Son la ubicación del jugador, representada como un número entero de 0 a 15  
- Como repaso del módulo 1, una política determinista podría tener este aspecto

| Observación | Acción |
|------|-------|
| 0 | 0 |
| 1 | 3 |
| 2 | 1 |
| 3 | 1 |
| ... | ... |
| 14 | 2 |
| 15 | 2 |

#### Políticas del lago aleatorio

Y una política no determinista podría tener el siguiente aspecto

| Observación | P(izquierda) | P(abajo) | P(derecha) | P(arriba) 
|------------|-------|-----------|---------|-------|
| 0 | 0 | 0.9 | 0.01 | 0.04 | 0.05
| 1 | 3 | 0.05 | 0.05 | 0.05 | 0.85
| ... | ... | ... | ...      | ...      | ...
| 15 | 2 | 0.0 | 0.0 | 0.99 | 0.01

Por cierto, esto no significa que RLlib aprenda esa tabla, pero podemos pensar en ella conceptualmente.

#### Políticas del lago aleatorio

- En el lago aleatorio, toda nuestra decisión debe basarse en la posición del jugador.
- A veces esto es suficiente: desde la posición 11, debes bajar.

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

- Pero ¿qué pasa con la posición 5, qué debes hacer a partir de ahí?
- Respuesta: _depende_. Si hay un agujero en la posición 9, no querrás bajar. Lo mismo ocurre con la 6 
- ¿Cómo puedo decidir _sin saber dónde están los agujeros_?

#### Estado contra observación, un resumen

- En el Módulo 1, definimos informalmente el estado como todo lo relacionado con el entorno.
- Aquí, eso incluiría la ubicación del jugador y los agujeros.
- La observación, en cambio, sólo codifica una parte del estado: en este caso, la ubicación del jugador.

#### ¿Observación = estado? Problema 1.

- Entonces, ¿por qué no establecer la observación en el estado? 
- Aquí hay dos problemas.
- Problema 1: cuando se despliega el sistema de RL, es posible que no se conozca todo el estado.
  - Ejemplo: en un sistema de recomendación, el agente (recomendador) no tiene acceso al estado de ánimo del usuario (parte del estado que afecta a los resultados)
  - En el aprendizaje supervisado, no queremos entrenar sobre características a las que no tendremos acceso en el despliegue
    - Del mismo modo, aquí la observación tiene que ser algo a lo que podamos acceder en el despliegue.

#### ¿Observación = estado? Problema 2.

- Problema 2: Puede ser difícil generalizar a partir de una observación realmente compleja.
  - Hay cientos de miles de estados posibles sólo en este pequeño juego de lago aleatorio de 4x4.
  - Demasiada información podría ser confusa para el agente o podría requerir cantidades irrazonables de datos (simulaciones) para darle sentido.

#### Observaciones sobre la codificación

- Parte de nuestro trabajo como practicante de RL es elegir una representación (o codificación) para la observación.
- A partir de la información que el jugador permitió conocer, encontrar una representación útil de lo que el jugador necesita saber.
- En nuestro caso, probaremos una aproximación: el jugador consigue "ver" si los 4 espacios adyacentes son agujeros o no.
- Codificaremos esto como 4 números binarios.

#### Observaciones sobre la codificación

```
.OO.
....
O.P.
...G
```

- En esta situación, no hay agujeros alrededor del jugador, por lo que el jugador "ve" `[0 0 0]` 
- En otras palabras, la observación aquí es `[0 0 0 0]`.

#### Observaciones sobre la codificación

```
.OO.
..P.
O.O.
...G
```

- Aquí, el jugador "ve" agujeros arriba y abajo, por lo que la observación es `[0 1 0 1]` (izquierda, abajo, derecha, arriba)

#### Observaciones sobre la codificación

¿Qué pasa con las aristas?

```
....
..OP
O.OO
...G
```

- Esta es nuestra elección al diseñar el espacio de observación.
- Elegiré representar "fuera de la red" como agujeros, lo que significa que fingimos que el lago tiene este aspecto:
 
```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

- Aquí, el jugador ve agujeros a la izquierda, abajo y derecha, por lo que la observación es `[1 1 1 0]` (izquierda, abajo, derecha, arriba)
- Sin embargo, puede haber enfoques mejores, porque caer en un agujero es peor (el episodio termina) que caminar por el borde (no pasa nada).

#### Codificar nuestras observaciones

- Ahora que tenemos un plan, ¿cómo modificamos el código?
- Como hemos estructurado nuestra clase para que tenga un método de `observación`, eso es todo lo que tenemos que modificar:

In [2]:
from envs_03 import RandomLake

class RandomLakeObs(RandomLake):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array (optional)
        return obs

- El código crea una variable `obs` en la que cada entrada es 1 si esa dirección sale de la arista **o** hay un agujero en ella.

In [3]:
# HIDDEN
import gym

#### Codificar nuestras observaciones

- Es necesario un cambio más en el código, que es el constructor donde se define el espacio de observación.
- Nuestras observaciones eran antes un número entero de 0 a 15, por lo que utilizamos

In [4]:
observation_space = gym.spaces.Discrete(16)

Y lo mismo para las acciones:

In [5]:
action_space = gym.spaces.Discrete(4)      

- Sin embargo, nuestras observaciones son ahora matrices de 4 números en lugar de un único número.
- Para indicarlo, utilizamos `gym.spaces.MultiDiscrete` en lugar de `gym.spaces.Discrete`.
- Multi, porque tenemos varios números, pero sigue siendo discreto, porque cada uno de los 4 números sólo puede tomar 2 valores posibles (0 ó 1).
- Este es el código:

In [6]:
class RandomLakeObs(RandomLake):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.MultiDiscrete([2,2,2,2])
        self.action_space = gym.spaces.Discrete(4)      

(Ten en cuenta que `gym` también tiene un tipo de espacio `MultiBinario`, pero actualmente no es compatible con RLlib)

#### Probando nuestro nuevo entorno

¡Vamos a probarlo!

In [7]:
# HIDDEN
import numpy as np
np.random.seed(42)

In [8]:
from envs_03 import RandomLakeObs

env = RandomLakeObs()
env.reset()

array([1, 1, 0, 1])

In [9]:
env.render()

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Aquí, vemos la observación esperada que indica "agujeros" a la izquierda, abajo y arriba.

Notas 

La izquierda y el arriba son los bordes del mapa, y el abajo es un agujero real.

#### Probando nuestro nuevo entorno

Intentemos dar un paso a la derecha:

In [10]:
env.step(2)

(array([0, 1, 0, 1]), 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [11]:
env.render()

🧊🧑🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Ahora vemos agujeros en las direcciones descendente y ascendente, de nuevo como se esperaba.

#### Formación con nuestras nuevas observaciones

- Nuestras nuevas observaciones parecen funcionar, pero ¿ayudan al agente a aprender?
- Recordemos que con nuestro espacio de observación `Discreto(16)` no pudimos conseguir mucho más que un 30% de éxito.
- Intentémoslo de nuevo:

In [12]:
# HIDDEN
from utils_03 import lake_default_config

In [13]:
ppo = lake_default_config.build(env=RandomLakeObs)

for i in range(8):
    ppo.train()

In [14]:
ppo.evaluate()["evaluation"]["episode_reward_mean"]

0.6420454545454546

- ¡Esto es mucho mejor que el ~30% que obteníamos antes!
- Lo cual tiene sentido... nuestro agente puede "ver" los agujeros ahora, en lugar de caminar a ciegas.

#### ¡Apliquemos lo que hemos aprendido!

## Analogía del aprendizaje supervisado: espacio de observación
<!-- multiple choice -->

En las diapositivas hemos cambiado el espacio de observación de nuestro agente y, como resultado, hemos conseguido mayores recompensas. ¿A qué aspecto del proceso de aprendizaje supervisado es más análogo?

- [x] Ingeniería de características | ¡Ya lo tienes! Nuestro espacio de observación actúa como el espacio de características sobre el que actúa nuestra política.
- [ ] Selección de modelos | No del todo. Pero, como veremos, ¡la selección de modelos también tiene cabida en la RL!
- [Ajuste de los hiperparámetros: no es así. Pero, como veremos, ¡el ajuste de hiperparámetros también tiene cabida en la RL!
- [ ] Selección de una función de pérdida

## Incluyendo la ubicación del jugador
<!-- multiple choice -->

En nuestra nueva representación de la observación, en realidad _eliminamos_ la ubicación del jugador de la observación y _sólo_ incluimos la presencia de los agujeros cercanos. Si quisiéramos un espacio de observación que incluyera tanto las paredes cercanas _como_ la ubicación del jugador, ¿cuál de los siguientes espacios de gimnasio podríamos utilizar?

- [ ] `gimnasio.espacios.Discreto(5)` | ¡Inténtalo de nuevo!
- [x] `gimnasio.espacios.MultiDiscreto([2,2,2,2,16])` | ¡Sí! Los 4 primeros números representan los agujeros, y el último número representa la ubicación del jugador.
- [ ] `gimnasio.espacios.MultiDiscreto([2,2,2,2]) + gimnasio.espacios.Discreto(16)` | Inténtalo de nuevo; lamentablemente no podemos añadir espacios de gimnasio.
- [ ] `gimnasio.espacios.MultiDiscreto([32,32,32,32])` | Esto podría funcionar, pero es una representación confusa/redundante.

## Manejo de los bordes
<!-- multiple choice -->

En las diapositivas hemos decidido tratar las aristas como agujeros. Recuerda esta imagen:

```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

Sin embargo, los bordes y los agujeros son en realidad diferentes entre sí: entrar en un borde no hace nada, mientras que entrar en un agujero hace que el episodio termine. Ésta podría ser una distinción importante, especialmente en una versión "resbaladiza" del entorno, en la que los resultados de las acciones no son deterministas 

Para abordar esta cuestión, decidimos cambiar el espacio de observación. El agente sigue "viendo" sólo los cuatro cuadrados que le rodean, pero ahora ve si cada cuadrado es un espacio vacío, un agujero o un borde. Para esta representación, ¿cuál de los siguientes espacios de observación del gimnasio podríamos utilizar?

- [ ] `gimnasio.espacios.MultiDiscreto([2,2,2,2,2,2,2,2])` | Inténtalo de nuevo. Recuerda que el agente sigue "viendo" sólo 4 casillas.
- [ ] `gym.spaces.MultiDiscrete([3,3,3,3,3,3,3,3])` | ¡Inténtalo de nuevo!
- [ ] `gimnasio.espacios.MultiDiscreto([2,2,2,2])` | Esto es lo mismo que el espacio anterior, pero hemos hecho un cambio.
- [x] `gimnasio.espacios.MultiDiscreto([3,3,3,3])` | ¡Ya lo tienes! Ahora hay 3 opciones posibles para lo que el agente puede "ver" en cada casilla.

In [15]:
# TODO / note to self
# query_policy(trainer, RandomLakeObs(), [1,1,1,1])
# shows that it wants to go up. this is because the above "hole" is probably an edge based on its learning. fascinating.

## Implementación de los bordes
<!-- coding exercise -->

El código siguiente muestra la función "observación" para el espacio de observación actual. Modifica el código para que utilice el nuevo espacio de observación, en el que 0 representa un espacio vacío, 1 representa un agujero y 2 representa una arista 

In [16]:
# EXERCISE

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[1 1 0 1]


In [17]:
# SOLUTION

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(2 if j==0 else self.holes[i,j-1]) # left
        obs.append(2 if i==3 else self.holes[i+1,j]) # down
        obs.append(2 if j==3 else self.holes[i,j+1]) # right
        obs.append(2 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[2 1 0 2]


## Lo que ve el agente
<!-- coding exercise -->

Con nuestra nueva codificación del espacio de observación, el agente sólo "ve" los 4 espacios que le rodean y sólo dispone de esta información para tomar sus decisiones. La celda de código siguiente crea una representación de lo que el agente "ve" mientras navega por el lago aleatorio. Puedes introducir acciones con el teclado escribiendo las palabras "izquierda", "abajo", "derecha" o "arriba" (o "l", "d", "r", "u" para abreviar) y la simulación te mostrará el resultado. (Escribe "quit" para salir.) Juega hasta que llegues a la meta. A medida que avanzas, intenta trazar un mapa del lago (quizás dibujando en un papel).

In [18]:
# TODO / NOTE:
# THIS EXERCISE DOES NOT HAVE A "solution"
# the code is here ONLY to help them answer the multiple choice

In [ ]:
# EXERCISE

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

Observation:
.O.
OP.
...



In [ ]:
# SOLUTION

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

#### ¿Qué aspecto tiene el lago?

Basándote en tus exploraciones, ¿cuál es el mapa correcto del lago en la pregunta anterior?

```
 (A) (B) (C) (D)
P..O P.OO P..O P.OO
..OO .OOO ..OO .OOO
O...     O...     O...     O..O
...G ...G ..OG ...G
```

- [x] (A)
- [ ] (B)
- [ ] (C)
- [ ] (D)

In [ ]:
# TODO
# could also considering showing a BAD environment encoding to contrast with this reasonable one, as in the next slide deck!